# Results

In [ ]:
# imports
import os
import yaml
import numpy as np
import pandas as pd
import theano
import lasagne
import loading
from training import *
from network import *
import architectures as arches
from scipy.stats import bayes_mvs, entropy, linregress, spearmanr

# aliases
L = lasagne.layers
nl = lasagne.nonlinearities
T = theano.tensor
bmvs = bayes_mvs

## Data Loading

In [ ]:
headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
paramsdir_ = os.path.join(headdir, 'Analysis/0_hvh/Params/nnets/')
datadir = os.path.join(headdir, 'Data/model input')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nnets')

data = loading.default_loader(os.path.join(datadir, '1-4 (no computer).csv'))
fake_data = loading.default_loader(os.path.join(datadir, 'fake news (with groups).csv'))
hvhdata = loading.default_loader(os.path.join(datadir, '0 (with groups).csv'))
df = hvhdata[0]
Xs = np.concatenate(hvhdata[2])
ys = np.concatenate(hvhdata[3])
Ss = np.concatenate(hvhdata[4])

defmod = np.loadtxt(os.path.expanduser('~/Downloads/loglik_hvh_final.txt')).reshape([40, 5])

with open('arch_specs.yaml') as archfile:
    arch_dict = yaml.load(archfile)

## Compile Results

In [ ]:
def unfreeze(net):
    # move this function onto Network class!
    for layer in L.get_all_layers(net.net):
        for param in layer.params:
            layer.params[param].add('trainable')
    net.params = L.get_all_params(net.net, trainable=True)
    
    return None

def compute_pretrained_results(archname, idx, test_data, fake=False):
    Xt, yt = test_data
    specs = arch_dict[archname]
    af = getattr(arches, arch_dict[archname]['type'])
    arch_func = lambda input_var: af(input_var, **specs['kwargs'])
    if fake:
        fname = '{} {} split fake data.npz'.format('fake_' + archname, idx)
        paramsdir = os.path.join(paramsdir_, 'fake_' + archname)
    else:
        fname = '{} {} split agg fit exp 1-4.npz'.format(archname.replace('_', ' '), idx)
        paramsdir = os.path.join(paramsdir_, archname[:-1])

    results_df = pd.DataFrame(index=np.arange(Xt.shape[0]), columns=[idx])

    net = Network(arch_func)
    
    net.load_params(os.path.join(paramsdir, fname))

    nlls = net.itemized_test_fn(Xt, yt)
    predictions = net.output_fn(Xt)
    results_df[idx] = nlls
    
    n_params = L.count_params(net.net)
    return results_df, predictions, n_params

def compute_tuned_results(archname, idx, test_idx, test_data):
    Xt, yt = test_data
    group_idx = (test_idx - 1) % 5 # fix eventually to take df/groupidx/selection passed independently?
    selection = df.loc[df['group']==(group_idx+1)].index.values
    results_df = pd.DataFrame(index=np.arange(Xt.shape[0]), columns=[idx])
    predictions_df = pd.DataFrame(index=selection, columns=np.arange(36))
    
    specs = arch_dict[archname]
    af = getattr(arches, arch_dict[archname]['type'])
    arch_func = lambda input_var: af(input_var, **specs['kwargs'])
    fname = '{} {} agg fit exp 1-4 {} tune fit exp 0.npz'.format(archname.replace('_', ' '), idx, test_idx)
    
    net = Network(arch_func)
    net.load_params(os.path.join(paramsdir_, archname[:-1], fname))
    
    nlls = net.itemized_test_fn(Xt[selection, :, :, :], yt[selection])
    predictions = net.output_fn(Xt[selection, :, :, :])
    predictions_df.loc[selection, :] = predictions
    results_df.loc[selection, idx] = nlls
    return results_df, predictions_df

In [ ]:
Xt, yt, _, _, _ = loading.unpack_data(df)
PTR = {}
TR = {}
param_counts = {}

for archname in arch_dict.keys():
    arch_dir = archname[:-1]

    if arch_dir not in os.listdir(paramsdir_):
        print("{} not started".format(archname[:-1]))
        continue
    
    files = os.listdir(os.path.join(paramsdir_, arch_dir))
    if not any(archname.replace('_', ' ') in f for f in files):
        print("{} not started".format(archname))
        continue
        
    if archname in ['deep_c1']:
        print(archname, 'not found')
        continue
        
    print(archname)

    pretrain_results = []
    pretrain_outputs = []

    for idx in range(5):
        results_df, predictions_df, n_params = compute_pretrained_results(archname, idx, (Xt, yt))
        pretrain_results.append(results_df)
        pretrain_outputs.append(predictions_df)

    pretrain_results = pd.concat(pretrain_results, axis=1)
    PTR[archname] = pretrain_results
    param_counts[archname] = n_params
    
    tune_results = []
    tune_predictions = []

    for idx in range(5):
        for test_idx in range(5):
            results_df, predictions_df  = compute_tuned_results(archname, idx, test_idx, (Xt, yt))
            tune_results.append(results_df)
            tune_predictions.append(predictions_df)

    tune_results = pd.concat(tune_results, axis=1, join='inner').stack().unstack()
    TR[archname] = trune_results
    
    pretrain_results.to_csv(os.path.join(resultsdir, 'pretrain {}.csv'.format(archname)))
    tune_results.to_csv(os.path.join(resultsdir, 'train {}.csv'.format(archname)))
    
    print("\n")

In [ ]:
pc_series = pd.Series(param_counts)
pc_series.to_csv(os.path.join(resultsdir, 'params per net.csv'))
F = pd.DataFrame(index=np.arange(len(pc_series.index)), columns=['net name'])
F['net name'] = pc_series.index
F['num params'] = pc_series.values

for k, v in PTR.items():
    for col in range(5):
        new_col_name = 'Pretrained {}'.format(col)
        idx = F['net name'] == k
        F.loc[idx, new_col_name] = v.pivot_table(index=df['subject'], values=col).mean()
        
for k, v in TR.items():
    for col in range(5):
        D = pd.read_csv(os.path.join(resultsdir, 'train {}.csv'.format(k))).drop('Unnamed: 0', axis=1)
        new_col_name = 'Tuned {}'.format(col)
        idx = F['net name'] == k
        F.loc[idx, new_col_name] = D.pivot_table(index=df['subject'], values=str(col)).mean()

tuned_cols = ['Tuned {}'.format(i) for i in range(5)]
untuned_cols = ['Pretrained {}'.format(i) for i in range(5)]
F['tuned mean'] = F[tuned_cols].mean(axis=1)
F['untuned mean'] = F[untuned_cols].mean(axis=1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

% matplotlib inline

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(8, 5))

for net_name in F['net name']:
    if 'deep' in net_name:
        color = 'green'
    elif 'regular' in net_name:
        color = 'red'
    else:
        color = 'purple'
        
    idx = F['net name'] == net_name
    x = F.loc[idx, 'num params'].values[0]
    y = F.loc[idx, 'tuned mean'].values[0]
    axes.plot(x, y, linestyle='none', marker='o', color=color)

plt.setp(axes, xlabel='# parameters', ylabel='NLL', xlim=[0, 10000])
sns.despine()

In [ ]:
F.to_csv(os.path.join(resultsdir, 'num params with nlls.csv'))

In [ ]:
FD = fake_data[0]
HH = hvhdata[0]
AD = data[0]

FD['position'] = FD['bp'] + FD['wp']
HH['position'] = HH['bp'] + FD['wp']
AD['position'] = AD['bp'] + AD['wp']

def entropy_zets(zets):
    z_ = np.histogram(zets, bins=np.arange(37), normed=True)[0]
    z_ = z_[z_ > 0]
    return -(z_ * np.log(z_)).sum()

FDpiv = FD.pivot_table(index='position', values='zet', aggfunc=entropy_zets)
HHpiv = HH.pivot_table(index='position', values='zet', aggfunc=entropy_zets)
ADpiv = AD.pivot_table(index='position', values='zet', aggfunc=entropy_zets)

In [ ]:
len(FDpiv), len(ADpiv), len(HHpiv)

In [ ]:
FDpiv.loc[FDpiv.values > 0].mean(), ADpiv.loc[ADpiv.values > 0].mean(), HHpiv.loc[HHpiv.values > 0].mean()

In [ ]:
FDpiv.mean(), ADpiv.mean(), HHpiv.mean()

In [ ]:
archname = 'h4'
Xt, yt, _, _, _ = loading.unpack_data(df)


pretrain_results = []
pretrain_outputs = []

for idx in range(5):
    results_df, predictions_df = compute_pretrained_results(archname, idx, (Xt, yt))
    pretrain_results.append(results_df)
    pretrain_outputs.append(predictions_df)

pretrain_results = pd.concat(pretrain_results, axis=1)

In [ ]:
pretrain_results.to_csv(os.path.join(resultsdir, 'pretrain {}.csv'.format(archname)))

for i, o in enumerate(pretrain_outputs):
    np.savetxt(os.path.join(resultsdir, 'network_predictions', '{}.csv'.format(i)), o, delimiter=',')

In [ ]:
pretrain_results['mean'] = pretrain_results.mean(axis=1)
pretrain_results.pivot_table(index=df['subject'], values='mean').mean()

In [ ]:
archname = 'h4'
Xt, yt, _, _, _ = loading.unpack_data(fake_data[0])

fake_results = []
fake_outputs = []

specs = arch_dict[archname]
af = getattr(arches, arch_dict[archname]['type'])
arch_func = lambda input_var: af(input_var, **specs['kwargs'])

for idx in range(5):
    fname = '{} {} split agg fit exp 1-4.npz'.format(archname, idx)
    paramsdir = os.path.join(paramsdir_, archname[:-1])

    net = Network(arch_func)
    net.load_params(os.path.join(paramsdir, fname))
    nlls = net.itemized_test_fn(Xt, yt)
    predictions = net.output_fn(Xt)


    fake_results.append(nlls)
    fake_outputs.append(predictions)

fake_results_df = pd.DataFrame(fake_results).T


In [ ]:
fake_results_df.pivot_table(index=fake_data[0]['subject']).mean().mean()

In [ ]:
archname = 'h4'
Xt, yt, _, _, _ = loading.unpack_data(df)

pretrain_results = []
pretrain_outputs = []

for idx in range(5):
#     print(idx)
    results_df, predictions_df, n_params = compute_pretrained_results(archname, idx, (Xt, yt), fake=True)
    pretrain_results.append(results_df)
    pretrain_outputs.append(predictions_df)

pretrain_results = pd.concat(pretrain_results, axis=1)

In [ ]:
pretrain_results.pivot_table(index=hvhdata[0]['subject']).mean().mean()

In [ ]:
train_nets = []
train_results = []
train_outputs = []

for idx in range(5):
    for test_idx in range(5):
        resdf, outputs, net = load_train_results(archname, archfunc, idx, test_idx)
        train_nets.append(net)
        train_results.append(resdf)
        train_outputs.append(outputs)

train_results = pd.concat(train_results, axis=1, join='inner').stack().unstack()
train_results.to_csv(os.path.join(resultsdir, 'train {}.csv'.format(archname)))

In [ ]:
for i in range(5):
    D_ = pd.concat(train_outputs[i:5+i]).sort_index()
    D_.to_csv(os.path.join(resultsdir, 'network_predictions', 'trained {}.csv'.format(i)), index=False, header=False)

In [ ]:
train_results['mean'] = train_results.mean(axis=1)
bmvs(train_results.pivot_table(index=df['subject'], values='mean').values)

# Per pieces

In [ ]:
def count_pieces(row):
    bp, wp = row[['bp', 'wp']]
    n_bp = np.array(list(bp)).astype(int).sum()
    n_wp = np.array(list(wp)).astype(int).sum()
    
    return n_bp + n_wp

df['np'] = df.apply(count_pieces, axis=1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('white')

% matplotlib inline

In [ ]:
train_results.mean()

In [ ]:
chancenll = lambda x: -np.log(1/(36-x))

In [ ]:
df['chancenll'] = chancenll(df['np'].values)
df['m'] = -(train_results.mean(axis=1).values - df['chancenll'])
np_v_m = df.pivot_table(index='np', values='m')
np_v_m.to_csv(os.path.join(resultsdir, 'num_pieces_vs_nll.csv'), header=False)
plt.plot(np_v_m)

plt.setp(plt.gca(), xlabel='N Pieces', ylabel='NLL relative to chance', ylim=[-.5, 2])

sns.despine()
plt.tight_layout()

In [ ]:
plt.plot(df.pivot_table(index='np', values='chancenll'))

# Response Times

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
plt.rc('text', usetex=True)

%matplotlib inline

In [ ]:
scatterkws = {
    'linestyle': 'none', 
    'marker': 'o', 'markerfacecolor': (.2, .2, .2), 'markeredgecolor': 'black', 
    'alpha': .3
}

histkws = {
    'edgecolor': 'white'
}

In [ ]:
def hicks_entropy(pred):
    H = pred * np.log2(1 / (pred + 1))
    return H.sum(axis=1)

In [ ]:
X, y, S, G, Np = loading.unpack_data(df)
df['mean corrected rt'] = 0
for subject in df['subject'].unique():
    fil = df['subject'] == subject
    df.loc[fil, 'mean corrected rt'] = df.loc[fil, 'rt'] - df.loc[fil, 'rt'].mean()

rt = df['mean corrected rt']

In [ ]:
# compute mean entropy for each test group
E = []
for split_idx in range(25):
    N = train_nets[split_idx]
    locs = np.where(G==(split_idx//5))[0]
    L = N.output_fn(X[locs, :, :, :])
    E.append(hicks_entropy(L))

for g in range(5):
    df.loc[df['group']==(g+1), 'entropy'] = np.array(E[g*5:(g+1)*5]).T.mean(axis=1)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(10, 6))

x = df['entropy']
y = np.log(df['rt']/1000)
axes.plot(x, y, **scatterkws)
lr = linregress(x, y)
pval = lr.pvalue if lr.pvalue >= .001 else .001
axes.text(.05, .05, r"r = {:.2f}, p $<$ {:.3f}".format(lr.rvalue, pval), transform=axes.transAxes, fontsize=14)
plt.setp(axes, xlabel=r"$\textrm{Entropy}$", ylabel=r'$\log{\textrm{Response time (s)}}$', ylim=[-5, 5])

sns.despine()

Hick's law holds (ish).

# Values

In [ ]:
gendata = pd.read_csv(
    os.path.join(headdir, 'Data/1_gen/Clean/_summaries/all_evals_model_input.csv'),
    names=['subject', 'color', 'bp', 'wp', 'zet', 'rt', 'val']
)
gendata['group'] = -1

X, y, S, G, Np = loading.unpack_data(gendata)

In [ ]:
N = train_nets[0]
logistic = lambda x: 1 / (1 + np.exp(-x))
zscore = lambda x: (x - x.mean()) / (x.std() / np.sqrt(x.shape[0]))
Vr = N.value_fn(X)
V = Vr.sum(axis=1)
Vl = 7*logistic(zscore(V))

V2 = np.zeros_like(V)
yz = np.zeros_like(y)
for subject in range(S.max()):
    V2[S==subject] = zscore(V[S==subject])
    yz[S==subject] = zscore(y[S==subject])
    
V2l = 7*logistic(V2)

In [ ]:
Vr = N.value_fn(X) - N.value_fn(X[:, ::-1, :, :])

In [ ]:
plt.hist(V, **histkws)
sns.despine()

In [ ]:
plt.hist(V2, **histkws) #, bins=np.arange(0, 8, .5), **histkws)
sns.despine()

In [ ]:
plt.plot(V, gendata['val'], **scatterkws)
print(linregress(V2, gendata['val']))
sns.despine()

In [ ]:
plt.plot(gendata['val'], gendata['zet'], **scatterkws)
print(linregress(gendata['zet'], gendata['val']))
sns.despine()

In [ ]:
plt.plot(zscore(V), yz, **scatterkws)
print(linregress(zscore(V), yz))
sns.despine()

In [ ]:
gendata['valhat'] = 6*logistic(V2) + 1
gendata['valhat'] = gendata['valhat'].map(int)
gendata['position'] = gendata['bp'] + gendata['wp']
gp = gendata.pivot_table(index='position', columns='zet', values='group', aggfunc=len, fill_value=0)
gvp = gendata.pivot_table(index='position', values='valhat')
gp['valhat'] = gvp.values
gp['valsum'] = gp[list(np.arange(1, 8, 1))].values.argmax(axis=1) + 1
gp.head()

In [ ]:
linregress(gp['valhat'], gp['valsum'])